In [1]:
import os
import time
import pickle
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [21]:
video_dir = '/mnt/disks/datastorage/videos/'
weights_dir = '/mnt/disks/datastorage/weights/'
#file = 'v_Shooting_g09_v1_a.mov'
#file = 'v_Shooting_g18_v1_c.MP4'
#file = 'v_Shooting_g08_v1_b.mov'
#file = 'v_Misc_g29_v1_b.mov'

#video_path = os.path.join(video_dir, 'fens', file)

# for live stream
#video_path = 'stream.sdp'

video_path = '/mnt/disks/datastorage/videos/cold_day/demos/shooting_ballroom_light6.mov'
video_path = '/mnt/disks/datastorage/videos/cold_day/demos/shooting_ballroom_dark2.mov'

nb_classes = 4
skip_num = 1
svm_model = '/home/mlp/two-stream-action-recognition/demos_svm.pkl'
prediction_path = '/mnt/disks/datastorage/predictions/'
flownet_weights = os.path.join(weights_dir, 'FlowNet2-CSS.pth.tar')
spatial_weights = os.path.join('/home/mlp/two-stream-action-recognition/record/spatial', 'model_best_FENS.pth.tar')
motion_weights = os.path.join('/home/mlp/two-stream-action-recognition/record/motion', 'model_best_FENS.pth.tar')
flownet_model = 'FlowNet2CSS'

In [22]:
!python3 -W ignore pipeline.py --stream $video_path \
                               --nb_classes $nb_classes \
                               --skip_frames $skip_num \
                               --svm $svm_model \
                               --save $prediction_path \
                               --model $flownet_model \
                               --optical_weights $flownet_weights \
                               --spatial_weights $spatial_weights \
                               --motion_weights $motion_weights \
                               #--spatial_only

Parsing Arguments
  [0.237s] fp16: False
  [0.237s] fp16_scale: 1024.0
  [0.237s] image_size: [224, 224]
  [0.237s] inference_size: [-1, -1]
  [0.237s] loss: L1Loss
  [0.237s] model: FlowNet2CSS
  [0.237s] model_batchNorm: False
  [0.237s] model_div_flow: 20.0
  [0.237s] motion_weights: /home/mlp/two-stream-action-recognition/record/motion/model_best_FENS.pth.tar
  [0.237s] nb_classes: 4
  [0.237s] number_gpus: 1
  [0.237s] optical_weights: /mnt/disks/datastorage/weights/FlowNet2-CSS.pth.tar
  [0.237s] rgb_max: 255.0
  [0.237s] save: /mnt/disks/datastorage/predictions/
  [0.237s] seed: 1
  [0.237s] skip_frames: 1
  [0.237s] spatial_only: False
  [0.237s] spatial_weights: /home/mlp/two-stream-action-recognition/record/spatial/model_best_FENS.pth.tar
  [0.237s] stream: /mnt/disks/datastorage/videos/cold_day/demos/shooting_ballroom_dark2.mov
  [0.237s] svm: /home/mlp/two-stream-action-recognition/demos_svm.pkl
  [0.237s] Operation finished

Loading inference models
Building spatial model


In [16]:
prediction_files = sorted(glob.glob(os.path.join(prediction_path, 'predictions_*.pkl')), reverse=True)

with open(prediction_files[0], 'rb') as pf:
    predictions = pickle.load(pf)
    predictions = np.array(predictions).squeeze()
    actions = np.argmax(predictions, axis=1) 

In [17]:
predictions_argmax = predictions.argmax(1)

for pred in list(zip(predictions, predictions_argmax)):
    print(pred)

(array([8.17785554e-07, 9.42683896e-07, 9.99998240e-01]), 2)
(array([8.41672826e-07, 9.71643347e-07, 9.99998187e-01]), 2)
(array([4.11238194e-06, 4.62595601e-06, 9.99991262e-01]), 2)
(array([1.51645804e-06, 1.72931056e-06, 9.99996754e-01]), 2)
(array([2.41973089e-06, 2.73051836e-06, 9.99994850e-01]), 2)
(array([4.42855499e-06, 4.94001584e-06, 9.99990631e-01]), 2)
(array([9.80352559e-04, 9.40709147e-04, 9.98078938e-01]), 2)
(array([1.15412373e-06, 1.32477711e-06, 9.99997521e-01]), 2)
(array([2.09858870e-06, 2.37987855e-06, 9.99995522e-01]), 2)
(array([0.00116802, 0.00104715, 0.99778483]), 2)
(array([1.22248648e-06, 1.39652609e-06, 9.99997381e-01]), 2)
(array([2.16719489e-06, 2.46172367e-06, 9.99995371e-01]), 2)
(array([9.14159397e-07, 1.05013693e-06, 9.99998036e-01]), 2)
(array([8.23712581e-07, 9.51176220e-07, 9.99998225e-01]), 2)
(array([9.02079799e-07, 1.04345940e-06, 9.99998054e-01]), 2)
(array([1.63847421e-06, 1.85800243e-06, 9.99996504e-01]), 2)
(array([9.49429426e-07, 1.08837045e-

In [6]:
predictions_extended = np.zeros(2*len(predictions_argmax))
for i in range(len(predictions_extended)):
    predictions_extended[i] = int(predictions_argmax[i//2])

In [7]:
action_dict = {
    'fens': {
        0: 'Theft',
        1: 'Assault',
        2: 'Shooting',
        3: 'No Action',
    },
    'demo': {
        0: 'Assault',
        1: 'Shooting',
        2: 'No Action',
    }
}

cap = cv2.VideoCapture(video_path)

frame_time = 0
frame_idx = 0
display = None
i = 0
while True:
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
    
    try:
        label = action_dict['demo'][actions[frame_idx]]
    except IndexError:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.putText(
        frame,
        label,
        (10, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (255, 255, 255),
        thickness=3
    )
    
    if i < 0:
        frame_time += time.time() - start_time
        frame_idx += 1
        i+=1
        continue
        
#     if frame_idx > 500:
#         break
        
    else:
        plt.axis('off')
        plt.imshow(frame)
        plt.show()
        clear_output(wait=True)

        frame_time += time.time() - start_time
        frame_idx += 1
cap.release()

frame_time /= frame_idx
print('Avg. frame processing time: {} s'.format(round(frame_time, 2)))